Import the twitter and MongoDB


In [2]:
from __future__ import unicode_literals
# https://github.com/sixohsix/twitter/tree/master
# Python Twitter Tool (Version 1.18.0)
from twitter import *
from twitter.stream import TwitterStream, Timeout, HeartbeatTimeout, Hangup
import time
import pymongo
from pymongo import MongoClient


Define the keys to access the developer option of our twitter

In [3]:
# Replace with your own key/secret
consumer_key = 'xtinDDzDZGttCY6CQcxoAU4La'
consumer_secret = 'qN4QUUil9Ah8JDMNJHSD92H0FlV71b5IFVTB3j0J4Df5AhAvOi'
resource_owner_key = '1172437840291979264-uUnfLvLWQVoebUZozGXT3C5Nfj9qbn'
resource_owner_secret = 'ztvUaUe2xQKTUmgdBVdO6fYArsJfuzpSZymrXx6efyt44'

set up the mongoDB database to store the tweets

In [4]:

# set-up the mongoDB to read json files
client = MongoClient('localhost', 27017)
db = client['twitter_hc_sample']
tweets_hc = db.tweets_hc

n = 0

The main loop in which we track the tweets with the specified hashtags and store them in our database. We have used a counter (n) to store 3000 tweets.

In [5]:
if __name__ == '__main__':
    while True:
        try:
            # When using twitter stream you must authorize.
            auth = OAuth(
                consumer_key=consumer_key,
                consumer_secret=consumer_secret,
                token=resource_owner_key,
                token_secret=resource_owner_secret
            )
            # Set location bounding box
            query_args = {'track': '#covid19,#covid-19,#coronavirus', 'stall_warnings':True}
            # Start authorization and filtering tweets
            stream = TwitterStream(auth=auth)
            tweet_iter = stream.statuses.filter(**query_args)

            # Iterate over the sample stream.
            for tweet in tweet_iter:
                # We should check the type of tweet.
                # It might be  a delete or data message.
                if tweet is None:
                    print("-- None --")
                elif tweet is Timeout:
                    print("-- Timeout --")
                elif tweet is HeartbeatTimeout:
                    print("-- Heartbeat Timeout --")
                elif tweet is Hangup:
                    print("-- Hangup --")
                else:
                    #print(tweet)
                    tweets_hc.insert_one(tweet)
                    n = n + 1
                    print(n)
                    if n==3001:
                        break;
                    
        except Exception as e:
            print('error (Follow): ' + str(e))
            # Let your application sleep in short time
            time.sleep(15)

error (Follow): Twitter sent status 401 for URL: 1.1/statuses/filter.json using parameters: (oauth_consumer_key=xtinDDzDZGttCY6CQcxoAU4La&oauth_nonce=3113594671549472232&oauth_signature_method=HMAC-SHA1&oauth_timestamp=1585652320&oauth_token=1172437840291979264-uUnfLvLWQVoebUZozGXT3C5Nfj9qbn&oauth_version=1.0&stall_warnings=True&track=%23covid19%2C%23covid-19%2C%23coronavirus&oauth_signature=LvyVKMCaLXIucpEW%2FLoza%2BPVnPw%3D)
details: <html>\n<head>\n<meta http-equiv="Content-Type" content="text/html; charset=utf-8"/>\n<title>Error 401 Unauthorized</title>
</head>
<body>
<h2>HTTP ERROR: 401</h2>
<p>Problem accessing '/1.1/statuses/filter.json'. Reason:
<pre>    Unauthorized</pre>
</body>
</html>



KeyboardInterrupt: 